<a href="https://colab.research.google.com/github/mlwright84/magictriangles/blob/main/Distributions_of_Entries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Distribution of Magic Triangle Entries

by Gabriel Hale, Bjorn Vogen, and Matthew Wright

This file contains the Python code to determine the distribution of integers in each position in 3-level and 4-level magic triangles. This was used to collect the data for Figure 6 in our *Magic Triangles* paper.

Note that an $n$-level triangular arrangement is stored as a list of $n^2$ entries, indexed left-to-right, bottom-to-top as in our paper (see Figure 2 in the paper). However, Python indexes start at 0, while indexes in the paper start at 1.

## 3-Level Triangles

This code collects the data to produce Figure 7 in the paper.

In [ ]:
# function to find all 3-level central hexagon solutions
def findCentralHexagons():
  sols = []

  # consider all triples of integers 1 <= a3 < a6 < a8 <= 9:
  for a3 in range(1,10):
    for a6 in range(a3+1,10):
      for a8 in range(a6+1,10):
        # there are six remaining numbers for the other positions
        remainingNums = [i for i in range(1,10) if (i != a3 and i != a6 and i != a8)]

        # iterate over triples of distinct indexes from 0, ..., 5
        for i in range(6):
          for j in range(6):
            if i == j:
              continue
            for k in range(6):
              if i == k or j == k:
                continue
              a2 = remainingNums[i]
              a4 = remainingNums[j]
              a7 = remainingNums[k]

              if a2 + a3 + a6 == 15 and a3 + a4 + a8 == 15 and a6 + a7 + a8 == 15:
                sols.append([a2, a3, a4, a6, a7, a8])
                print([a2, a3, a4, a6, a7, a8])

  return sols

# function to compute fequencies of each integer in each position
def computeFrequencies(centralHexs):
  corners = [0]*9
  borders = [0]*9
  interior = [0]*9

  for sol in centralHexs:
    a2 = sol[0]
    a3 = sol[1]
    a4 = sol[2]
    a6 = sol[3]
    a7 = sol[4]
    a8 = sol[5]

    borders[a3-1] += 6
    borders[a6-1] += 6
    borders[a8-1] += 6

    interior[a2-1] += 6
    interior[a4-1] += 6
    interior[a7-1] += 6

    remaining = [i for i in range(1,10) if i not in sol]
    for k in remaining:
      corners[k-1] += 6

  return corners, borders, interior

Now we can find all 16 central hexagon solutions for 3-level magic triangles:

In [ ]:
sols = findCentralHexagons()
sols

[9, 1, 8, 5, 4, 6]
[9, 1, 6, 5, 2, 8]
[9, 2, 8, 4, 6, 5]
[9, 2, 7, 4, 5, 6]
[9, 2, 5, 4, 3, 8]
[8, 2, 7, 5, 4, 6]
[8, 2, 6, 5, 3, 7]
[8, 2, 4, 5, 1, 9]
[7, 2, 5, 6, 1, 8]
[8, 3, 7, 4, 6, 5]
[7, 3, 4, 5, 2, 8]
[6, 4, 3, 5, 2, 8]
[6, 4, 2, 5, 1, 9]
[5, 4, 3, 6, 1, 8]
[4, 5, 3, 6, 2, 7]
[4, 5, 2, 6, 1, 8]


Next we compute the number of times that each integer is in the corner, border, and interior positiosn for all 96 3-level magic triangles:

In [ ]:
corners, borders, interior = computeFrequencies(sols)

In [ ]:
print(corners)
print(borders)
print(interior)

[54, 18, 54, 18, 0, 18, 54, 18, 54]
[12, 42, 12, 42, 72, 42, 12, 42, 12]
[30, 36, 30, 36, 24, 36, 30, 36, 30]


In [ ]:
# function to print coordinate lists for the plot in the paper
def printCoordStr(counts):
  s = "{"
  for i,c in enumerate(counts):
    s += "(" + str(i+1) + "," + str(c) + ") "
  s += "}"
  return s

In [ ]:
print(printCoordStr(corners))
print(printCoordStr(borders))
print(printCoordStr(interior))

{(1,54) (2,18) (3,54) (4,18) (5,0) (6,18) (7,54) (8,18) (9,54) }
{(1,12) (2,42) (3,12) (4,42) (5,72) (6,42) (7,12) (8,42) (9,12) }
{(1,30) (2,36) (3,30) (4,36) (5,24) (6,36) (7,30) (8,36) (9,30) }


## 4-Level Triangles

This code collects the data to produce Figure 8 in the paper.

Exhaustive search for 4-level magic triangles: creates a list `magicConfigs` with entries as follows:

[corners, B, A, C, a_10, a_14, a_2, a_6]

a_14 < a_2 < a_6


In [ ]:
# exhaustive search for 4-level magic triangles: creates a list `magicConfigs` with entries as follows:
#   [corners, B, A, C, a_10, a_14, a_2, a_6]
#   entries in corners, A, B, and C are sorted in ascending order, and also a_14 < a_2 < a_6
def count4LevelTriangles():
  global numChecked, numFound, interiorConfigs
  numChecked = 0
  numFound = 0
  interiorConfigs = []

  # choose 3 of 16 numbers for the corners
  for i in range(1,17):
    for j in range(i+1,17):
      for k in range(j+1, 17):
        interiorNums = [n for n in range(1,17) if (n != i and n != j and n != k) ]
        #print(interiorNums)

        count4LevelInteriors(interiorNums, [i, j, k])

        print(f"finished [{i}, {j}, {k}]")

  print(f"checked {numChecked} cases")
  print(f"found {numFound} magic triangles")

# function that finds interior configurations (everything except the corner entries)
def count4LevelInteriors(nums, corners):
  global numChecked, numFound, interiorConfigs

  # choose indexes for region I
  for i in range(13):
    for j in range(i+1,13):
      for k in range(j+1,13):
        regIsum = nums[i] + nums[j] + nums[k]
        numsNotI = [n for n in nums if (n != nums[i] and n != nums[j] and n != nums[k]) ]

        # choose indexes for region II
        for p in range(10):
          for q in range(p+1,10):
            for r in range(q+1,10):
              regIIsum = numsNotI[p] + numsNotI[q] + numsNotI[r]
              numsNotII = [n for n in numsNotI if (n != numsNotI[p] and n != numsNotI[q] and n != numsNotI[r]) ]

              # choose indexes for region III
              for s in range(7):
                for t in range(s+1,7):
                  for u in range(t+1,7):
                    regIIIsum = numsNotII[s] + numsNotII[t] + numsNotII[u]
                    final4 = [n for n in numsNotII if (n != numsNotII[s] and n != numsNotII[t] and n != numsNotII[u]) ]

                    # choose middle number (a) in the triangle
                    # this determines the remaining 3 numbers, which we specify to be CCW increasing from top (b < c < d)
                    for v in range(4):
                      a = final4[v]
                      final3 = final4[:v] + final4[v+1:]  # this is [b, c, d]
                      
                      # compute sums
                      magicNum = 68 - a
                      if (regIsum + regIIIsum + final3[0] == magicNum) and (
                          regIsum + regIIsum + final3[1] == magicNum) and (
                          regIIsum + regIIIsum + final3[2] == magicNum):
                        # found a magic triangle!
                        numFound += 1
                        config = [ corners, [nums[i], nums[j], nums[k]], [numsNotI[p], numsNotI[q], numsNotI[r]],
                                  [numsNotII[s], numsNotII[t], numsNotII[u]], a, final3[0], final3[1], final3[2]]
                        magicConfigs.append(config)
                        #print( config )

                      numChecked += 1


Now find all 4-level magic triangles. The following code cell takes about 25 minutes to run.

In [ ]:
magicConfigs = []
count4LevelTriangles()

finished [1, 2, 3]
finished [1, 2, 4]
finished [1, 2, 5]
finished [1, 2, 6]
finished [1, 2, 7]
finished [1, 2, 8]
finished [1, 2, 9]
finished [1, 2, 10]
finished [1, 2, 11]
finished [1, 2, 12]
finished [1, 2, 13]
finished [1, 2, 14]
finished [1, 2, 15]
finished [1, 2, 16]
finished [1, 3, 4]
finished [1, 3, 5]
finished [1, 3, 6]
finished [1, 3, 7]
finished [1, 3, 8]
finished [1, 3, 9]
finished [1, 3, 10]
finished [1, 3, 11]
finished [1, 3, 12]
finished [1, 3, 13]
finished [1, 3, 14]
finished [1, 3, 15]
finished [1, 3, 16]
finished [1, 4, 5]
finished [1, 4, 6]
finished [1, 4, 7]
finished [1, 4, 8]
finished [1, 4, 9]
finished [1, 4, 10]
finished [1, 4, 11]
finished [1, 4, 12]
finished [1, 4, 13]
finished [1, 4, 14]
finished [1, 4, 15]
finished [1, 4, 16]
finished [1, 5, 6]
finished [1, 5, 7]
finished [1, 5, 8]
finished [1, 5, 9]
finished [1, 5, 10]
finished [1, 5, 11]
finished [1, 5, 12]
finished [1, 5, 13]
finished [1, 5, 14]
finished [1, 5, 15]
finished [1, 5, 16]
finished [1, 6, 7]
fin

In [ ]:
len(magicConfigs)

184056

In [ ]:
len(magicConfigs)*(6**4)

238536576

In [ ]:
magicConfigs[:20]

[[[1, 2, 10], [4, 9, 13], [5, 7, 12], [6, 8, 11], 3, 14, 15, 16],
 [[1, 2, 10], [4, 9, 13], [5, 8, 11], [6, 7, 12], 3, 14, 15, 16],
 [[1, 2, 10], [4, 9, 13], [6, 7, 11], [5, 8, 12], 3, 14, 15, 16],
 [[1, 2, 10], [5, 8, 13], [4, 9, 11], [6, 7, 12], 3, 14, 15, 16],
 [[1, 2, 10], [5, 8, 13], [6, 7, 11], [4, 9, 12], 3, 14, 15, 16],
 [[1, 2, 10], [5, 9, 12], [4, 7, 13], [6, 8, 11], 3, 14, 15, 16],
 [[1, 2, 10], [5, 9, 12], [6, 7, 11], [4, 8, 13], 3, 14, 15, 16],
 [[1, 2, 10], [6, 7, 13], [4, 8, 12], [5, 9, 11], 3, 14, 15, 16],
 [[1, 2, 10], [6, 7, 13], [4, 9, 11], [5, 8, 12], 3, 14, 15, 16],
 [[1, 2, 10], [6, 7, 13], [5, 8, 11], [4, 9, 12], 3, 14, 15, 16],
 [[1, 2, 10], [6, 8, 12], [4, 7, 13], [5, 9, 11], 3, 14, 15, 16],
 [[1, 2, 10], [6, 8, 12], [4, 9, 11], [5, 7, 13], 3, 14, 15, 16],
 [[1, 2, 10], [6, 9, 11], [4, 7, 13], [5, 8, 12], 3, 14, 15, 16],
 [[1, 2, 10], [6, 9, 11], [4, 8, 12], [5, 7, 13], 3, 14, 15, 16],
 [[1, 2, 10], [6, 9, 11], [5, 7, 12], [4, 8, 13], 3, 14, 15, 16],
 [[1, 2, 1

Now compute the frequencies of each integer in each position.

In [ ]:
# function to compute frequencies for 4-level triangle
def computeFrequencies4(configs):
  corner = [0]*16
  border = [0]*16
  interior = [0]*16
  center = [0]*16

  for sol in configs:
    # corners
    entries = sol[0]
    for i in entries:
      corner[i-1] += 1

    # borders
    entries = sol[1] + sol[2] + sol[3]
    for i in entries:
      border[i-1] += 1

    # interior
    interior[sol[5]-1] += 1
    interior[sol[6]-1] += 1
    interior[sol[7]-1] += 1

    # center
    center[sol[4]-1] += 1

  return corner, border, interior, center

In [ ]:
corner, border, interior, center = computeFrequencies4(magicConfigs)

Here are the frequencies. Observe that the border frequencies are exactly three times the corner frequencies, and the interior frequencies are exactly three times the center frequencies.

In [ ]:
print(corner)
print(border)
print(interior)
print(center)

[34536, 34436, 34565, 34422, 34523, 34532, 34491, 34579, 34579, 34491, 34532, 34523, 34422, 34565, 34436, 34536]
[103608, 103308, 103695, 103266, 103569, 103596, 103473, 103737, 103737, 103473, 103596, 103569, 103266, 103695, 103308, 103608]
[34434, 34734, 34347, 34776, 34473, 34446, 34569, 34305, 34305, 34569, 34446, 34473, 34776, 34347, 34734, 34434]
[11478, 11578, 11449, 11592, 11491, 11482, 11523, 11435, 11435, 11523, 11482, 11491, 11592, 11449, 11578, 11478]


In [ ]:
print([3*i for i in corner])

[103608, 103308, 103695, 103266, 103569, 103596, 103473, 103737, 103737, 103473, 103596, 103569, 103266, 103695, 103308, 103608]


In [ ]:
print([3*i for i in center])

[34434, 34734, 34347, 34776, 34473, 34446, 34569, 34305, 34305, 34569, 34446, 34473, 34776, 34347, 34734, 34434]
